# RWKV v5-wavenet 1B5 / embedding init-range 1e-01 / 16k
This model is based on the RWKV standard 1B5 model

- 24 layers
- 2048 embedding size

Going through the modified memory training for v5 models, across various initial embedding model weights

**Note:** This project assumes you have the rwkv-infctx conda env setup

# Basic Setup

In [ ]:
# First lets setup the various directories, and init the model
!mkdir -p ../../../../model/
!mkdir -p ../../../../datapath/
!mkdir -p ../../../../checkpoint/

In [ ]:
# Additional dependencies for eval stuff
!pip install -q aiocsv aiofiles

In [ ]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
EMBED_SCALE=0.1

WANDB_PREFIX=f"v5base-1B5-E{EMBED_SCALE}"

EMBED_SCALE_LABEL=str(EMBED_SCALE).replace(".", "_")
FILENAME_PREFIX=f"v5base-1B5-E{EMBED_SCALE_LABEL}"

# WAVENET LAYERS settings
RWKV_WAVENET_LAYERS=1

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)
print("RWKV_WAVENET_LAYERS:", RWKV_WAVENET_LAYERS)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))
INFERENCE_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("INFERENCE_DIR:", INFERENCE_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

In [ ]:
# Init the model
!cd "{TRAINER_DIR}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python3 ./init_model.py \
        --n_layer 24 --n_embd 2048 \
        --emb-scale "{EMBED_SCALE}" \
        --vocab_size neox --skip-if-exists \
        "../model/L24-D2048-E{EMBED_SCALE_LABEL}-neox-v5base-init.pth"

## Enwiki Stage 1 : Foundation 4k model training

In [ ]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/v5base-1B5-enwiki-4k.yaml"

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-1B5-enwiki-4k.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki-4k Foundation (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-enwiki-4k/" \
        --model.load_model="../model/L24-D2048-E{EMBED_SCALE_LABEL}-neox-v5base-init.pth" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/{FILENAME_PREFIX}-enwiki-4k/last.ckpt" "../model/{FILENAME_PREFIX}-enwiki-4k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-enwiki-4k.pth"

In [ ]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python3 dragon_test.py "../model/{FILENAME_PREFIX}-enwiki-4k.pth" "cuda fp32"

In [ ]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-enwiki-4k.pth"

## Enwiki Stage 1 : Foundation 16k model training

In [ ]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/v5base-1B5-enwiki-16k.yaml"

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-1B5-enwiki-16k.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki-16k Foundation (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-enwiki-16k/" \
        --model.load_model="../model/{FILENAME_PREFIX}-enwiki-4k.pth" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=4

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/{FILENAME_PREFIX}-enwiki-16k/last.ckpt" "../model/{FILENAME_PREFIX}-enwiki-16k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-enwiki-16k.pth"

In [ ]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python3 dragon_test.py "../model/{FILENAME_PREFIX}-enwiki-16k.pth" "cuda fp32"

In [ ]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-enwiki-16k.pth"

# Enwiki Stage 2 : Basic Instruct Tuning

In [ ]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/v5base-1B5-enwiki-instruct.yaml"

In [ ]:
# Start the instruct finetuning
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-1B5-enwiki-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki-Instruct (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-enwiki-instruct/" \
        --model.load_model="../model/{FILENAME_PREFIX}-enwiki-16k.pth" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/{FILENAME_PREFIX}-enwiki-instruct/last.ckpt" "../model/{FILENAME_PREFIX}-enwiki-instruct.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-enwiki-instruct.pth"

In [ ]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python3 dragon_test.py "../model/{FILENAME_PREFIX}-enwiki-instruct.pth" "cuda fp32"

In [ ]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-enwiki-instruct.pth"

## Tune 1 : Simple Memory instruct finetuning

- Tune 1: Low ctx size (512), Training with only the input masked. This does very limited memory training, and is used primarily to train the instruction set.

In [ ]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

# We do a strong bias for smaller word count, to teach the concept from scratch
# so that the model can learn the function. 
#
# Note that all document samples, are randomized between the target word count, 
# to half of the target word count.
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-2-count.jsonl  2  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-5-count.jsonl  5  5000 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-10-count.jsonl 10 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-15-count.jsonl 15 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-20-count.jsonl 20 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-25-count.jsonl 25 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-40-count.jsonl 40 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-50-count.jsonl 50 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-60-count.jsonl 80 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-80-count.jsonl 80 2500 &

# With a slight mix of the larger word count
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-100-count.jsonl 100 2500 &
python ../memory_script/gen_limited_segmented_jsonl.py ../dataset/word-200-count.jsonl 200 2500 &

wait
echo "## Done ##"

ls -alh ../dataset/

In [ ]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-1B5-mem-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Instruct (train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-instruct/" \
        --model.load_model="../model/{FILENAME_PREFIX}-enwiki-instruct.pth" \
        --model.ctx_len=512 \
        --model.bptt_learning_range=1

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-instruct/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-instruct.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-instruct.pth"

In [ ]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-instruct.pth"

## Tune 2 : Low ctx size (512), memory training

- Tune 2: Low ctx size (512), Training with instruction & input masked. This forces the actual memory training on the output tokens.

In [ ]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We switch over to fully masked instruct+input, to properly learn the memorization task
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl  2  5000 &
for i in {5..95..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 5000 & 
done
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-100-count.jsonl 100 5000 &
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-200-count.jsonl 200 5000 &

#
# We mixin the shuffled word list, so that we ensure all words / tokens are learned
# however this might intrduce an exclusion bias (if seen this word, never repeat it), 
# so we limit the mixture of this data samples
#
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-10-count.jsonl 10 20 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-15-count.jsonl 15 20 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-25-count.jsonl 25 30 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-50-count.jsonl 50 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-75-count.jsonl 75 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-100-count.jsonl 100 50 &
python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-200-count.jsonl 200 50 &

wait
echo "## Done ##"

ls -alh ../dataset/

In [ ]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-1B5-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-512 (train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-512/" \
        --model.lr_init=5e-4 \
        --model.lr_final=4e-4 \
        --data.max_token_size=512 \
        --model.ctx_len=512 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-instruct.pth"

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-512/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-512.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-512.pth"

In [ ]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-512.pth"

## Tune 3 : Low ctx size (1024), memory training

- Tune 3: Low ctx size (1024), Scaling up !

In [ ]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for lower word count - and shift the focus upwards
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 400 &
for i in {5..45..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 400 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 10 & 
done

#
# Ramping up the 50+ - 510 words dataset
# 
for i in {50..550..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 800 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

In [ ]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-1B5-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-1k (train-ctx=1k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-1k/" \
        --model.lr_init=4e-4 \
        --model.lr_final=2e-4 \
        --data.max_token_size=1024 \
        --model.ctx_len=1024 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-512.pth"

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-1k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-1k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

In [ ]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

## Tune 4 : Low ctx size (2048), memory training

- Tune 4: Low ctx size (2048), Scaling up !

In [ ]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for lower word count - and shift the focus upwards
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 100 &
for i in {5..100..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 100 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the 105+ - 1050 words dataset
# 
for i in {105..2000..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 200 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

In [ ]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-1B5-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-2k (train-ctx=2k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-2k/" \
        --model.lr_init=3e-4 \
        --model.lr_final=1e-4 \
        --data.max_token_size=2048 \
        --model.ctx_len=2048 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-2k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-2k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-2k.pth"

In [ ]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-2k.pth"

## Tune 5 : Ramping up the ctx size (4096), memory training

- Tune 5: Mid ctx size (4096), Scaling up!

In [ ]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for < 50 words - and shift the focus upwards
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 100 &
for i in {5..500..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 100 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the 50+ - 2100 words dataset
# 
for i in {505..4000..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 200 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -alh ../dataset/

In [ ]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-1B5-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-4k (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-4k/" \
        --model.lr_init=3e-4 \
        --model.lr_final=1e-4 \
        --data.max_token_size=4096 \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-1k.pth"

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-4k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-4k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-4k.pth"

In [ ]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-4k.pth"

## Tune 6 : Ramping up the ctx size (8192), memory training

- Tune 6: Large ctx size (8192), Scaling up!

In [ ]:
%%script bash

########################################
# Generate the required jsonl dataset
########################################

# Reset the dataset dir
mkdir -p ../dataset
rm -rf ../dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

#
# We reduce the training set for < 50 words - and shift the focus upwards
# (aka 50-100 token * 2 : ~100 - 250 token ctx len)
#
python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/word-2-count.jsonl 2 50 &
for i in {5..1000..5} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 50 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 1 & 
done

#
# Ramping up the 50+ - 4200 words dataset
# 
for i in {1100..8000..100} 
do
    python ../memory_script/gen_limited_prompt_completion_jsonl.py ../dataset/gen-word-$i-count.jsonl $i 2000 & 
    python ../memory_script/shuffle_limited_prompt_completion_jsonl.py ../dataset/shuffle-word-$i-count.jsonl $i 20 & 
done

wait
echo "## Done ##"

ls -lh ../dataset/

In [ ]:
# Start the finetune model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/v5base-1B5-mem-template.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Tune ctx-8k (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-mem-ctx-8k/" \
        --model.lr_init=3e-4 \
        --model.lr_final=1e-4 \
        --data.max_token_size=8192 \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=2 \
        --model.load_model="../model/{FILENAME_PREFIX}-mem-ctx-4k.pth"

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/{FILENAME_PREFIX}-mem-ctx-8k/last.ckpt" \
        "../model/{FILENAME_PREFIX}-mem-ctx-8k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-mem-ctx-8k.pth"

In [ ]:
# Lets do a quick memory test
!export RWKV_WAVENET_LAYERS="{RWKV_WAVENET_LAYERS}" && \
        python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/{FILENAME_PREFIX}-mem-ctx-8k.pth"